# Introduction to Accessing Blockchain Data
A Code Along to understand and access basic blockchain data. We will use a public hosted blockchain node to access network, account, block and transaction details. 

**Author: Harish Raisinghani, DevRel @ Covalent**. https://twitter.com/harish_yvr

## Web3.py Setup

We will be using [Web3.py](https://web3py.readthedocs.io/en/stable/quickstart.html), a comprehensive Python library for interacting with the programmable [Ethereum](https://ethereum.org/) blockchain, in this Code Along. 

We install the library using `pip`:

In [1]:
pip install web3

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 570 kB 1.8 MB/s eta 0:00:01
     |████████████████████████████████| 360 kB 5.8 MB/s eta 0:00:01
     |████████████████████████████████| 90 kB 10.7 MB/s eta 0:00:01
     |████████████████████████████████| 397 kB 9.6 MB/s eta 0:00:01
     |████████████████████████████████| 120 kB 11.4 MB/s eta 0:00:01
  Using cached requests-2.28.2-py3-none-any.whl (62 kB)
     |████████████████████████████████| 102 kB 18.5 MB/s eta 0:00:01
  Using cached charset_normalizer-3.1.0-cp39-cp39-macosx_10_9_x86_64.whl (124 kB)
     |████████████████████████████████| 61 kB 29.5 MB/s eta 0:00:01
     |████████████████████████████████| 60 kB 6.4 MB/s eta 0:00:01
     |████████████████████████████████| 48 kB 15.3 MB/s eta 0:00:01
     |████████████████████████████████| 120 kB 14.2 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 11.4 MB/s eta 0:00:01
     |███████████████████████████

## Connect to a Hosted Node Provider

In order to read data from blockchains, you can either:

1. Run your own node on that blockchain, download the entire ledger, execute the transactions and fetch the data from your node. 

2. Use a hosted node provider who manage the infrastructure for you

> Deeper dive: Set up your own Ethereum node following [this guide.](https://ethereum.org/en/developers/docs/nodes-and-clients/run-a-node/)

3. Use a data provider like [Covalent](https://www.covalenthq.com/platform/) that offers APIs to read data from blockchains

Examples of hosted Ethereum node providers are listed here: https://ethereumnodes.com/ . This is just a small set but does cover many of the major providers. 

In this Code Along, we will use the [Pocket Network](https://pokt.network/
 ) hosted node provider which offers a comprehensive [list of public nodes](https://docs.pokt.network/use/public-rpc/). Note the URL for accessing the Ethereum Mainnet node: https://eth-rpc.gateway.pokt.network

 The following code configures our `web3.py` library to use the Pocket Network as our node provider:

In [2]:
from web3 import Web3

eth_mainnet_provider = 'https://eth-rpc.gateway.pokt.network'

w3_mainnet = Web3(Web3.HTTPProvider(eth_mainnet_provider))

# Check that we have a web3 object
w3_mainnet


## JSON-RPC
Assuming that you are familiar with the `JSON` data format, `JSON-RPC` is a remote procedure call (RPC) protocol which follows a [specification](https://www.jsonrpc.org/specification) defining standards around data structures and processing in distributed computing.

The Ethereum protocol implements the [**Ethereum JSON-RPC**](https://ethereum.github.io/execution-apis/api-documentation/) specification - a standard collection of methods that all Ethereum clients implement so that applications can expect some consistency regardless of the node or client implementation.  

Examples of Ethereum JSON-RPC methods include:

* `eth_chainId`: Returns the unique chain ID of the current blockchain network
* `eth_gasPrice`: Returns the current price per gas in wei
* `eth_blockNumber`: Returns the sequence number of the most recent block
* `eth_getBalance`: Returns the balance of the account of a given address

For the full list of Ethereum JSON-RPC methods, see: https://ethereum.github.io/execution-apis/api-documentation/.


## JSON-RPC & `web3.eth`

The Web3.py library uses the `web3.eth` API object for interacting with the Ethereum blockchain and thus provides a mapping to Ethereum JSON-RPC methods. 

![Web3.py JSON-RPC Setup](https://i.imgur.com/QOqFqJ3l.png)

The `web3.eth` object exposes both *properties* and *methods* to interact with the JSON-RPC APIs under the `eth_` namespace.

For example the following shows JSON-RPC methods mapped to to `web3.eth` properties and methods:

### Properties:
*   `eth_chainId` -> `web3.eth.chain_id`
*   `eth_gasPrice` -> `web3.eth.gas_price`

### Methods:
*   `eth_blockNumber` -> `web3.eth.get_block_number()`
*   `eth_getBalance` -> `web3.eth.get_balance(<address>)`


See the full list of in `web3.eth` mappings [here](https://web3py.readthedocs.io/en/stable/web3.eth.html#methods).

Let us try a few examples:

In [ ]:
w3_mainnet.eth.chain_id

1

Chain IDs are unique blockchain identifiers for Ethereum-compatible blockchain networks. For example, Ethereum Mainnet has a chain ID of 1 while Polygon Mainnet has a chain ID of 137. 

You can find the full list of Ethereum-compatible blockchain's chain IDs (including both mainnets and testnets) at: https://chainlist.org/

In [ ]:
w3_mainnet.eth.gas_price

19909730434

In [ ]:
w3_mainnet.eth.get_block_number()

15454222

In [3]:
balance_wei = w3_mainnet.eth.get_balance('demo.eth')
print(balance_wei)

563418429638025147


A couple of key things to note:

1. Ethereum blockchain values, such as the gas price or account balance are commonly denominated in unit called `wei`, which is the smallest unit of ether (ETH) equal to `10^-18`. You can find the full `wei` to `ether` table at: https://gwei.io/. 

2. The `eth.get_balance()` takes either an alphanumeric account address OR an [Ethereum Name Service](https://ens.domains/) domain (which gets resolved to the underlying account address). 

> Deeper dive - search and set your own ENS domain (fee required): https://app.ens.domains/

So to convert our balance from `wei` to `ether` we can divide `balance_wei` by `10^18`)

In [ ]:
balance = balance_wei / (10**18)
print(balance)

0.6456238284254397


OR we can use the built in `.from_wei()` method and specify the `ether` unit:

In [4]:
balance = w3_mainnet.from_wei(balance_wei, 'ether')
print(balance)

0.563418429638025147


So this is interesting to explore blockchain data effectively like command line, but wouldn't it be nice to have a front-end that we can use? Enter the **Block Explorer**. 

## Block Explorers

![Etherscan](https://info.etherscan.com/content/images/2019/12/image-1.png)

A front-end that allows users to explore blockchain data without having to manually make API calls with code are called **block explorers**. The most popular Ethereum block explorer is [Etherscan](https://etherscan.io/). There, we can compare the results to the `web3.eth` API calls made above. 

> Q) What are some other block explorers for Ethereum mainnet and testnets?

## Transactions & Transaction Hashes

The state of the Ethereum blockchain changes as a result of **transactions** or data (signed messages) that is committed to the blockchain by an account. Think of blockchains like Ethereum as global [singleton](https://en.wikipedia.org/wiki/Singleton_pattern) state machines where transactions are the driver changing the machine's state. Smart contracts or programs on blockchains *do not run on their own*. Everything starts with a transaction.

> Deeper dive - learn more about different [types of transactions](https://www.covalenthq.com/docs/unified-api/guides/understanding-blockchain-transactions-with-covalent/)

Transactions on the blockchain are recorded as transaction hashes. In this Code Along, we are simply accessing existing transaction data through transaction hashes, not creating new transactions (for a later Code Along).


### Get a Transaction by Hash
We can use the Block Explorer to find a transaction hash and then we can see all the key details of a transaction by using the [`eth_getTransactionByHash`](https://web3py.readthedocs.io/en/stable/web3.eth.html#web3.eth.Eth.get_transaction) JSON-RPC method. 

Let us take the following transaction hash: [0x7bebda592bfb8923b80f5498c58a8a5abf2c689feeafd8a6c46fc8626b249c56](https://etherscan.io/tx/0x7bebda592bfb8923b80f5498c58a8a5abf2c689feeafd8a6c46fc8626b249c56) which is an example of a basic ETH transfer/payment:

In [5]:
w3_mainnet.eth.get_transaction('0x7bebda592bfb8923b80f5498c58a8a5abf2c689feeafd8a6c46fc8626b249c56')

AttributeDict({'accessList': [],
 'blockHash': HexBytes('0xe8ed5b0aaabc311e05b89c76ea236f91d8107529d7aa5d8084d151adac1a8566'),
 'blockNumber': 15454264,
 'chainId': 1,
 'from': '0x347334073a11B74C0029C7B9Cf43a56dCB0d7c1C',
 'gas': 21000,
 'gasPrice': 22035434010,
 'hash': HexBytes('0x7bebda592bfb8923b80f5498c58a8a5abf2c689feeafd8a6c46fc8626b249c56'),
 'input': '0x',
 'maxFeePerGas': 24709787665,
 'maxPriorityFeePerGas': 1500000000,
 'nonce': 19,
 'r': HexBytes('0xfaf61893b260fb3522f328e53be69703ffa829cf2ca24954c84a5a7d365d4c9b'),
 's': HexBytes('0x3ea579fcf16eefb9447c788f20751c9c2b3be5fb63b76fc95c4af00e88ef031c'),
 'to': '0x31532aA0137b1c1E1b9164C3Fe11634F4218c5b7',
 'transactionIndex': 174,
 'type': 2,
 'v': 1,
 'value': 50000000000000000})

We see the following key details:
- `nonce`: a sequence number issued by the sending Ethereum account indicating the number of previous transactions to prevent message replay
- `blockNumber`: the block where this transaction was placed in
- `from`: the sending Ethereum account
- `to`: the receiving Ethereum account
- `value`: the amount of ETH transferred (in Wei)
- `input`: any data sent with the transaction
- `gas`: the amount of gas provided by the sending Ethereum account
- `gasPrice`: the price in Wei per unit of gas provided by the sending Ethereum account
- `r`, `s`, `v` make up the components of the sender's encrypted digital signature

This is effectively the structure of any Ethereum-based transaction. In this example, there is a `value` amount but no `input` data so this represents a payment transaction using the native currency of the blockchain (i.e. ETH on Ethereum).

If the transaction involves a smart contract function, the details of the function being invoked are passed in the `input` field. 


### Get a Transaction by Block
We can also get the number of transactions for a given block and then pass an index value to get a specific transaction within a block. For example:

In [6]:
w3_mainnet.eth.get_block_transaction_count(15454264)

205

In [7]:
# Fetch the first transaction in block 15454264 using an index value of 0
w3_mainnet.eth.get_transaction_by_block(15454264,0)

AttributeDict({'accessList': [],
 'blockHash': HexBytes('0xe8ed5b0aaabc311e05b89c76ea236f91d8107529d7aa5d8084d151adac1a8566'),
 'blockNumber': 15454264,
 'chainId': 1,
 'from': '0xEA674fdDe714fd979de3EdF0F56AA9716B898ec8',
 'gas': 250000,
 'gasPrice': 21535434010,
 'hash': HexBytes('0x88824a6cfaf3f02edb0d36efe46f918789dbc3ec26c2ef36b973e1a5ac9b0fa7'),
 'input': '0x',
 'maxFeePerGas': 32946580402,
 'maxPriorityFeePerGas': 1000000000,
 'nonce': 44876480,
 'r': HexBytes('0x8794af6856ae07fbb843ab867431a4f7afce319300a056996288526499d0f538'),
 's': HexBytes('0x482f0916443ac3d7da0f758f8d3abd8d21f7c9a7c3ad2d034639ab8f7d8250b3'),
 'to': '0x587Ea10544Ad3B22f08232c1146cC61c54566c36',
 'transactionIndex': 0,
 'type': 2,
 'v': 1,
 'value': 82393545439714279})

> Deeper dive - learn more about [gas fees](https://ethereum.org/en/developers/docs/gas/#pre-london) including `maxFeePerGas` and `maxPriorityFeePerGas` values.

## Summary
So we have covered how to access basic blockchain data - blockchain details, account balances and individual and block transactions. 

In the next Code Along, we look at data generated by smart contracts and in particular, custom fungible tokens (alt coins such as stablecoins, utility coins, asset coins, etc..) which are referred to as ERC20 standard tokens. This means that all tokens created under the ERC20 standard use the same smart contract interface and hence can be traded seamlessly. 